# Введение в MapReduce модель на Python


In [212]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [213]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [214]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [215]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [216]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [217]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [218]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [219]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [220]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [221]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [222]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [223]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [224]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [225]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [226]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.2041677803627377),
 (1, 1.2041677803627377),
 (2, 1.2041677803627377),
 (3, 1.2041677803627377),
 (4, 1.2041677803627377)]

## Inverted index 

In [227]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [228]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [229]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [230]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18)]),
 (1, [('is', 18), ('what', 10)])]

## TeraSort

In [231]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.02864986366543376),
   (None, 0.03915008333828651),
   (None, 0.05100790444317893),
   (None, 0.10137151779218667),
   (None, 0.10949087373136146),
   (None, 0.2418379970174902),
   (None, 0.2582099736711718),
   (None, 0.2716784104017891),
   (None, 0.2957814180435243),
   (None, 0.3077480290527732),
   (None, 0.36714985134319955),
   (None, 0.43361765926454376),
   (None, 0.43767000424729086),
   (None, 0.45301443224637616),
   (None, 0.4780750968058636),
   (None, 0.49556361452821773)]),
 (1,
  [(None, 0.5262876475542675),
   (None, 0.5280682289449264),
   (None, 0.6401131511619635),
   (None, 0.6891958623659773),
   (None, 0.7080477973445839),
   (None, 0.7614958241288134),
   (None, 0.8095261291710496),
   (None, 0.8232460879874129),
   (None, 0.8945010257548972),
   (None, 0.9341595436974924),
   (None, 0.9898492416034287),
   (None, 0.9924338247268286),
   (None, 0.9961520307599907),
   (None, 0.9966812014696435)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [232]:
# Имитация входных данных
input_numbers = [3, 41, 52, 26, 38, 57, 9, 49]

# RECORDREADER
def RECORDREADER():
    for index, value in enumerate(input_numbers):
        yield (index, value)

# MAP функция
def MAP(_, value):
    yield (None, value)  # Используем None как ключ, так как нам не важна группировка по ключу

# REDUCE функция
def REDUCE(_, values: Iterator):
    max_value = max(values)  # Находим максимум среди всех значений
    yield (None, max_value)

# Исполнение MapReduce
def MapReduce(RECORDREADER, MAP, REDUCE):
    map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
    shuffle_output = groupbykey(map_output)
    reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
    return list(reduce_output)

# Выполнение MapReduce алгоритма
max_value_output = MapReduce(RECORDREADER, MAP, REDUCE)
print(max_value_output)

[(None, 57)]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [233]:
input_numbers = [10, 20, 30, 40, 50]

# RECORDREADER
def RECORDREADER():
    for index, value in enumerate(input_numbers):
        yield (index, value)

# MAP функция
def MAP(_, value):
    yield (None, value)  # Используем None как ключ, так как нам не важно разделение на группы

# REDUCE функция
def REDUCE(_, values: Iterator):
    total_sum = 0
    count = 0
    for value in values:
        total_sum += value
        count += 1
    average = total_sum / count  # Вычисляем среднее значение
    yield (None, average)

# Выполнение MapReduce
def MapReduce(RECORDREADER, MAP, REDUCE):
    map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
    shuffle_output = groupbykey(map_output)
    reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
    return list(reduce_output)

# Выполнение MapReduce алгоритма для вычисления среднего
average_output = MapReduce(RECORDREADER, MAP, REDUCE)
print(average_output)

[(None, 30.0)]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [234]:
import random
from itertools import chain

# Функция для выравнивания списка списков
def flatten(lst):
    return list(chain.from_iterable(lst))

# Функция MAP, которая возвращает пару (ключ, значение)
def MAP(input_data):
    return (1, input_data)

# Функция для группировки по ключу
def groupbykey(items):
    sorted_items = sorted(items, key=lambda x: x[0])
    grouped = {}
    for key, value in sorted_items:
        if key in grouped:
            grouped[key].append(value)
        else:
            grouped[key] = [value]
    return [(k, list(v)) for k, v in grouped.items()]

# Функция REDUCE, вычисляющая среднее значение
def REDUCE(key, values):
    total_sum = 0
    total_count = 0
    for value in values:
        total_sum += value
        total_count += 1
    average = total_sum / total_count if total_count > 0 else 0
    return average

# Функция, имитирующая чтение записей (в данном случае генерирует случайные числа)
def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

# Генерируем данные и применяем функцию MAP
mapped_data = list(map(lambda x: MAP(x), RECORDREADER(10)))

print("Mapped Data:", mapped_data)

# Группируем данные по ключу
shuffle_output = groupbykey(mapped_data)

print("Shuffled Data:", shuffle_output)

# Применяем функцию REDUCE
reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))

print("Reduce Output:", reduce_output)

Mapped Data: [(1, 44), (1, 29), (1, 66), (1, 67), (1, 81), (1, 40), (1, 9), (1, 67), (1, 71), (1, 47)]
Shuffled Data: [(1, [44, 29, 66, 67, 81, 40, 9, 67, 71, 47])]
Reduce Output: [52.1]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [235]:
def INPUTFORMAT():
    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield (
            (docid, line)
            for docid, document in enumerate(documents[i:i + split_size])
            for lineid, line in enumerate(document.split('\n'))
        )


def MAP(docId, line):
    for word in line.split():
        if word:  # Проверяем, что слово не пустое
            yield (word, 1)


def REDUCE(key, _):
    yield (key, None)  # Возвращаем только ключи, исключая дубликаты


def groupbykey_distributed(map_output, PARTITIONER):
    partitions = [{} for _ in range(reducers)]
    for key, value in map_output:
        partition = PARTITIONER(key)
        partitions[partition][key] = None  # Игнорируем значение, сохраняем только ключ
    return [(i, partition.keys()) for i, partition in enumerate(partitions)]


def PARTITIONER(key):
    return hash(key) % reducers


def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE):
    map_output = flatten([MAP(docId, line) for docId, line in flatten(INPUTFORMAT())])
    shuffle_output = groupbykey_distributed(map_output, PARTITIONER)
    reduce_output = [(partition_id, list(flatten(REDUCE(key, None) for key in keys))) for partition_id, keys in
                     shuffle_output]
    return reduce_output


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)
print(partitioned_output)

[(0, [('it', None), ('a', None), ('banana', None)]), (1, [('is', None), ('what', None)])]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [236]:
import random

# Функция, генерирующая список случайных трёхмерных кортежей
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

# Предикат для функции MAP
def C(t):
    return sum(t) > 200

# Функция MAP
def MAP(el_list):
    mapped_result = {t: [t] for t in el_list if C(t)}
    return mapped_result.items()

# Функция REDUCE
def REDUCE(mapped_items):
    return [key for key, _ in mapped_items]

# Генерация записей
record = RECORDREADER(10)
print("Generated records:", record)

# Разбиваем записи на части
part_count = 5
record_partitional = [record[i:i + part_count] for i in range(0, len(record), part_count)]

# Применяем функции MAP и REDUCE
reduced_output = REDUCE([item for sublist in map(MAP, record_partitional) for item in sublist])
print("Reduced output:", reduced_output)

Generated records: [(69, 47, 83), (100, 54, 12), (53, 2, 0), (82, 8, 98), (63, 94, 21), (60, 13, 24), (60, 56, 30), (38, 100, 70), (91, 85, 74), (4, 7, 61)]
Reduced output: [(38, 100, 70), (91, 85, 74)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [237]:
# Определяем множество индексов для проекции
S = {0, 2}  # Проекция на первый и третий элементы трёхмерного кортежа

def MAP(t):
    # Проекция кортежа на множество атрибутов S
    res = tuple(t[i] for i in range(len(t)) if i in S)
    return (res, res)

def REDUCE(key, values):
    # Возвращаем только ключ, так как значения дублируют ключ
    return key

def RECORDREADER(count):
    # Генерация списка случайных трёхмерных кортежей
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable):
    grouped = {}
    for k, v in iterable:
        grouped.setdefault(k, []).append(v)
    return list(grouped.items())

# Генерация и обработка записей
record = RECORDREADER(5)
map_output = [MAP(x) for x in record]
shuffle_output = group_by_key(map_output)
reduce_output = [REDUCE(key, values) for key, values in shuffle_output]

# Вывод результатов
print("Original records:", record)
print("Map output:", map_output)
print("Shuffle (group by key) output:", shuffle_output)
print("Reduce output:", reduce_output)


Original records: [(9, 77, 25), (3, 79, 78), (79, 15, 15), (44, 58, 94), (50, 45, 76)]
Map output: [((9, 25), (9, 25)), ((3, 78), (3, 78)), ((79, 15), (79, 15)), ((44, 94), (44, 94)), ((50, 76), (50, 76))]
Shuffle (group by key) output: [((9, 25), [(9, 25)]), ((3, 78), [(3, 78)]), ((79, 15), [(79, 15)]), ((44, 94), [(44, 94)]), ((50, 76), [(50, 76)])]
Reduce output: [(9, 25), (3, 78), (79, 15), (44, 94), (50, 76)]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [238]:
import random
from typing import List, Tuple, Iterator

TupleType = Tuple[int, int, int]

# Функция MAP
def MAP(t: TupleType) -> Tuple[TupleType, TupleType]:
    return (t, t)

# Функция REDUCE
def REDUCE(key: TupleType, values: Iterator[TupleType]) -> Tuple[TupleType, TupleType]:
    return (key, key)

# Функция для генерации случайных кортежей
def RECORDREADER(count: int) -> List[TupleType]:
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

# Функция для группировки по ключу
def group_by_key(iterable: List[Tuple[TupleType, TupleType]]) -> List[Tuple[TupleType, List[TupleType]]]:
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return list(t.items())

# Генерация и обработка данных
record = RECORDREADER(3)
map_output = [MAP(x) for x in record]

shuffle_output = list(group_by_key(map_output))

reduce_output = [REDUCE(key, values)[0] for key, values in shuffle_output]

print("Record:\n", record)
print("Map Output:\n", map_output)
print("Shuffle Output:\n", shuffle_output)
print("Reduce Output:\n", reduce_output)

Record:
 [(6, 32, 13), (65, 96, 68), (20, 99, 78)]
Map Output:
 [((6, 32, 13), (6, 32, 13)), ((65, 96, 68), (65, 96, 68)), ((20, 99, 78), (20, 99, 78))]
Shuffle Output:
 [((6, 32, 13), [(6, 32, 13)]), ((65, 96, 68), [(65, 96, 68)]), ((20, 99, 78), [(20, 99, 78)])]
Reduce Output:
 [(6, 32, 13), (65, 96, 68), (20, 99, 78)]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [239]:
import random
from typing import Tuple, List

# Определение типа кортежа
TupleType = Tuple[int, int]

# Функция MAP
def MAP(t: TupleType) -> Tuple[TupleType, TupleType]:
    return (t, t)

# Функция REDUCE
def REDUCE(key: TupleType, values: List[TupleType]) -> Tuple[TupleType, TupleType]:
    if len(values) > 1:  # Проверяем, что элемент присутствует в обоих наборах
        return (key, key)

# Функция для генерации случайных кортежей
def RECORDREADER(count: int) -> List[TupleType]:
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

# Функция для группировки по ключу
def group_by_key(iterable: List[Tuple[TupleType, TupleType]]) -> List[Tuple[TupleType, List[TupleType]]]:
    grouped = {}
    for key, value in iterable:
        grouped.setdefault(key, []).append(value)
    return list(grouped.items())

# Генерация двух наборов данных
record1 = RECORDREADER(3)
record2 = RECORDREADER(3)

# Объединение записей из обоих наборов
combined_records = record1 + record2

# Применение функции MAP и группировка результатов
map_output = [MAP(x) for x in combined_records]
shuffle_output = list(group_by_key(map_output))

# Применение функции REDUCE и фильтрация ненужных результатов
reduce_output = [REDUCE(*item) for item in shuffle_output if len(item[1]) > 1]

print("Record1:\n", record1)
print("Record2:\n", record2)
print("Reduce Output:\n", [item for item in reduce_output])


Record1:
 [(0, 3), (1, 0), (0, 3)]
Record2:
 [(3, 3), (1, 2), (3, 0)]
Reduce Output:
 [((0, 3), (0, 3))]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [240]:
import random
from collections import defaultdict
from typing import List, Dict, Tuple

# Константы для обозначения принадлежности кортежа к множеству
SET_R = 0
SET_S = 1

# Создание случайного элемента с указанием его принадлежности к множеству
def create_random_element() -> Tuple[Tuple[int, int], int]:
    element = (random.randint(1, 4), random.randint(1, 4))
    set_indicator = random.choice([SET_R, SET_S])
    return (element, set_indicator)

# Функция чтения данных, имитирующая получение набора данных
def fetch_data(sample_size: int) -> List[Tuple[Tuple[int, int], int]]:
    return [create_random_element() for _ in range(sample_size)]

# Функция отображения, присваивающая каждому элементу его множество
def map_element(input_element: Tuple[Tuple[int, int], int]) -> Tuple[Tuple[int, int], int]:
    return input_element

# Функция свёртки, выбирающая элементы, принадлежащие только первому множеству
def filter_unique(key: Tuple[int, int], belonging: List[int]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    if belonging.count(SET_R) == 1 and SET_S not in belonging:
        return (key, key)

# Организация элементов по ключам
def organize_by_key(elements: List[Tuple[Tuple[int, int], int]]) -> Dict[Tuple[int, int], List[int]]:
    organization = defaultdict(list)
    for element_key, relation in elements:
        organization[element_key].append(relation)
    return organization

# Процесс MapReduce
data_samples = fetch_data(5)
mapped_samples = [map_element(sample) for sample in data_samples]
organized_data = organize_by_key(mapped_samples)
filtered_results = [filter_unique(k, v) for k, v in organized_data.items() if filter_unique(k, v) is not None]

# Вывод результатов
print("Generated Samples:\n", data_samples)
print("Mapped Samples:\n", mapped_samples)
print("Organized Data:\n", dict(organized_data))
print("Filtered (Unique) Results:\n", filtered_results)

Generated Samples:
 [((4, 3), 0), ((1, 3), 1), ((2, 1), 1), ((1, 3), 0), ((4, 3), 1)]
Mapped Samples:
 [((4, 3), 0), ((1, 3), 1), ((2, 1), 1), ((1, 3), 0), ((4, 3), 1)]
Organized Data:
 {(4, 3): [0, 1], (1, 3): [1, 0], (2, 1): [1]}
Filtered (Unique) Results:
 []


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [241]:
import random
from collections import defaultdict
from typing import Dict, List, Tuple, Iterable

# Идентификаторы для отношений R и S
REL_R = 'R'
REL_S = 'S'

# Генерация случайного кортежа для отношений R и S
def generate_random_tuple(is_relation_r: bool) -> Tuple[Tuple[int, int], str]:
    values = (random.randint(0, 3), random.randint(0, 3))
    relation = REL_R if is_relation_r else REL_S
    return (values, relation)

# Чтение записей, аналогичное RECORDREADER
def fetch_records(total: int) -> List[Tuple[Tuple[int, int], str]]:
    return [generate_random_tuple(i < total // 2) for i in range(total)]

# MAP функция, трансформирующая входные данные
def MAP(record: Tuple[Tuple[int, int], str]) -> Tuple[int, Tuple[str, int]]:
    ((a, b), relation) = record
    return (b, (relation, a)) if relation == REL_R else (a, (relation, b))

# REDUCE функция, обрабатывающая сгруппированные данные
def REDUCE(key: int, grouped_values: Iterable[Tuple[str, int]]) -> List[Tuple[int, int, int]]:
    r_values = [value for relation, value in grouped_values if relation == REL_R]
    s_values = [value for relation, value in grouped_values if relation == REL_S]
    return [(r, key, s) for r in r_values for s in s_values]

# Группировка по ключу, аналогичная group_by_key
def group_by_key(mapped: List[Tuple[int, Tuple[str, int]]]) -> Dict[int, List[Tuple[str, int]]]:
    grouped = defaultdict(list)
    for key, value in mapped:
        grouped[key].append(value)
    return grouped

# Пример использования
data = fetch_records(7)  # Генерация случайных записей
mapped = [MAP(item) for item in data]  # Применение MAP
grouped = group_by_key(mapped)  # Группировка по ключу
reduced = [result for key, values in grouped.items() for result in REDUCE(key, values)]  # Применение REDUCE

print("Raw Data:\n", data)
print("Mapped Data:\n", mapped)
print("Grouped Data:\n", dict(grouped))
print("Reduced Results:\n", reduced)

Raw Data:
 [((2, 1), 'R'), ((3, 0), 'R'), ((1, 3), 'R'), ((1, 2), 'S'), ((0, 3), 'S'), ((0, 0), 'S'), ((3, 3), 'S')]
Mapped Data:
 [(1, ('R', 2)), (0, ('R', 3)), (3, ('R', 1)), (1, ('S', 2)), (0, ('S', 3)), (0, ('S', 0)), (3, ('S', 3))]
Grouped Data:
 {1: [('R', 2), ('S', 2)], 0: [('R', 3), ('S', 3), ('S', 0)], 3: [('R', 1), ('S', 3)]}
Reduced Results:
 [(2, 1, 2), (3, 0, 3), (3, 0, 0), (1, 3, 3)]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [242]:
import random
from typing import List, Tuple, Dict, Callable

# Генерация случайного трехэлементного кортежа
def get_random_tuple() -> Tuple[int, int, int]:
    return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

# Имитация чтения данных, генерируя список случайных кортежей
def RECORDREADER(count: int) -> List[Tuple[int, int, int]]:
    return [get_random_tuple() for _ in range(count)]

# Функция MAP, которая преобразует входные кортежи
def MAP(t: Tuple[int, int, int]) -> Tuple[int, int]:
    return (t[0], t[1])

# Агрегирующая функция
def theta_aggregate(values: List[int], mode: str) -> int:
    if mode == 'sum':
        return sum(values)
    elif mode == 'max':
        return max(values)
    elif mode == 'min':
        return min(values)
    elif mode == 'avg':
        return sum(values) / len(values) if values else 0

# Функция REDUCE
def REDUCE(key: int, values: List[int], operation: str) -> Tuple[int, int]:
    return (key, theta_aggregate(values, operation))

# Группировка элементов по ключу
def group_by_key(iterable: List[Tuple[int, int]]) -> Dict[int, List[int]]:
    grouped = {}
    for key, value in iterable:
        if key not in grouped:
            grouped[key] = [value]
        else:
            grouped[key].append(value)
    return grouped

# Обработка записей
record = RECORDREADER(10)
map_output = [MAP(x) for x in record]
grouped_output = group_by_key(map_output)

# Выполнение агрегации
aggregations = ['sum', 'max', 'min', 'avg']
for agg in aggregations:
    reduce_output = [REDUCE(key, values, agg) for key, values in grouped_output.items()]
    print(f"{agg.capitalize()} Reduce Output:\n{reduce_output}")

Sum Reduce Output:
[(2, 9), (0, 4), (1, 5)]
Max Reduce Output:
[(2, 3), (0, 3), (1, 3)]
Min Reduce Output:
[(2, 2), (0, 0), (1, 2)]
Avg Reduce Output:
[(2, 2.25), (0, 1.0), (1, 2.5)]


### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [243]:
# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [244]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)  # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J, K)


def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])


def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(small_mat.shape[0]):
        yield ((i, k), small_mat[i, j] * w)


def REDUCE(key, values):
    (i, k) = key
    result = sum(values)
    yield ((i, k), result)

Проверьте своё решение

In [245]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [246]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [247]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
    yield from (((0, i, j), value) for i, row in enumerate(small_mat) for j, value in enumerate(row))
    yield from (((1, j, k), value) for j, row in enumerate(big_mat) for k, value in enumerate(row))


def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    key = j if mat_num == 0 else i
    yield (key, (mat_num, i if mat_num == 0 else j, v1))


def REDUCE_JOIN(key, values):
    for v1 in values:
        if v1[0] == 0:
            for v2 in values:
                if v2[0] == 1:
                    yield ((v1[1], v2[1]), v1[2] * v2[2])

def MAP_MUL(k1, v1):
    yield k1, v1


def REDUCE_MUL(key, values):
    yield (key, sum(values))


def GET_JOINED():
    yield from joined

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) 

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [248]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in
            enumerate(partitions)]


def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers


def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(lambda reduce_partition: (
    reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))),
                         reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(
        sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs


I = 3
J = 4
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
reference_solution = np.matmul(small_mat, big_mat)


def INPUTFORMAT():
    first_mat = [((0, i, j), value) for i, row in enumerate(small_mat) for j, value in enumerate(row)]
    second_mat = [((1, j, k), value) for j, row in enumerate(big_mat) for k, value in enumerate(row)]
    return [first_mat, second_mat]


def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    key = j if mat_num == 0 else i
    yield (key, (mat_num, i if mat_num == 0 else j, v1))


def REDUCE_JOIN(key, values):
    for v1 in values:
        if v1[0] == 0:
            for v2 in values:
                if v2[0] == 1:
                    yield ((v1[1], v2[1]), v1[2] * v2[2])


def GET_JOINED():
    yield from (j[1] for j in joined)


def MAP_MUL(k1, v1):  #+
    yield k1, v1


def REDUCE_MUL(key, values):
    yield (key, sum(values))


maps = 8
reducers = 4
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = list(map(lambda x: (x[0], list(x[1])), partitioned_output))

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = list(map(lambda x: (x[0], list(x[1])), mul_output))

solution = [v for p in pre_result for v in p[1]]

np.allclose(reference_solution, asmatrix(solution))  # should return true

172 key-value pairs were sent over a network.
480 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [249]:
def flatten(nested_iterable):
  # Генератор для раскрытия вложенной структуры данных
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  # Группировка по ключу для итерируемой последовательности
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  # Распределенная группировка по ключу
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  # Функция для вычисления партиций с использованием хеш-функции
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  # Функция для выполнения распределенной обработки MapReduce
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} пар ключ-значение было отправлено по сети.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  # Функция для форматирования входных данных в MapReduce
  first_mat = [((0, i, j), small_mat[i,j]) for i in range(small_mat.shape[0]) for j in range(small_mat.shape[1])]

  second_mat = [((1, j, k), big_mat[j,k]) for j in range(big_mat.shape[0]) for k in range(big_mat.shape[1])]

  # Разбиваем первую матрицу на части
  split_size = int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  # Разбиваем вторую матрицу на части
  split_size = int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

def MAP_JOIN(k1, v1):
    # MAP-функция для присоединения (join) данных
    mat_num, i, j = k1
    key = j if mat_num == 0 else i
    yield (key, (mat_num, i if mat_num == 0 else j, v1))

def REDUCE_JOIN(key, values):
    # REDUCE-функция для присоединения (join) данных
    for v1 in values:
        if v1[0] == 0:
            for v2 in values:
                if v2[0] == 1:
                    yield ((v1[1], v2[1]), v1[2] * v2[2])

def GET_JOINED():
  # Генератор для извлечения данных после присоединения
  yield from (j[1] for j in joined)

def MAP_MUL(k1, v1): #+
    # MAP-функция для умножения
    yield k1, v1

def REDUCE_MUL(key, values):
    # REDUCE-функция для умножения
    yield (key, sum(values))

maps = 8
reducers = 4
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = list(map(lambda x: (x[0], list(x[1])), partitioned_output))

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = list(map(lambda x: (x[0], list(x[1])), mul_output))

solution = [v for p in pre_result for v in p[1]]

np.allclose(reference_solution, asmatrix(solution)) # должен вернуть True

126 пар ключ-значение было отправлено по сети.
240 пар ключ-значение было отправлено по сети.


True